# 第5回 その1: 線形判別分析
3つのクラスの3教科テスト点数に対して線形判別分析を行ってみましょう。


## ステップ0: Google Driveのマウントと作業フォルダへの移動  
Google Drive に配置したデータを読み込むための準備です。  
詳細については第二回の 02_01_graph.ipynb を参照してください。  

ここでは"マイドライブ/情報管理/05"を作業フォルダとします。 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# フォルダの移動には"%cd"を使用します。
# 作業フォルダへ移動
%cd /content/drive/'My Drive'/情報管理/05/
# 現在のフォルダの中身を表示
%ls

`sankyouka.csv`というデータが表示されていることを確認してください。

## ステップ1: データの読み込み
まずは必要ライブラリをインポートします。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

`sankyouka.csv` を読み込みます。  

In [ ]:
# pandas の関数 read_csv を用いた csvファイル読み込み
csv_data = pd.read_csv('sankyouka.csv', encoding='SHIFT-JIS')
# データの前半部(.headで取得できる)のみ表示
display(csv_data.head())

# numpy用データ(ndarray型) に変換する。
X = csv_data.loc[:,'国語':].to_numpy()
y = csv_data.loc[:,'クラス']

# データのサンプル数と次元数を得る。
(num_samples, num_dimensions) = np.shape(X)
print('Information of scores (X)')
print('  Nunber of samples: ' + str(num_samples))
print('  Number of dimensions: ' + str(num_dimensions))

# クラスの種類を得る。
# unique(): pandasデータリストから，重複する数値を消す関数（[1,2,1,3,3,4] -> 重複数字を消して[1,2,3,4]となる）
classes = y.unique()
# クラス数を得る。
num_classes = np.size(classes)

print('Classes:' + str(classes))
print('Number of classes: ' + str(num_classes))

このデータは学生300名の3教科（国語，英語，数学）の得点を記録したものです。  
得点情報は変数 `X` に格納されました。  
よって`X`のサンプル数は300，次元数は3です。  

また各学生は3種類のクラス（0, 1, 2）のどこかに属しています。  
学生ごとのクラス番号は y に格納されました。  
すなわち，クラス数 num_classes = 3 です。

3次元データなので，以下のようにして，一応プロットすることが可能です。

In [ ]:
color_set = ['b', 'r', 'g']

from mpl_toolkits.mplot3d import axes3d
from ipywidgets import interact

def polyscatter(elevation):
  fig = plt.figure(figsize=(8,8))
  ax = fig.add_subplot(111, projection="3d")
  for n in range(num_classes):
    ax.scatter(X[y==n,0], X[y==n,1], X[y==n,2], color=color_set[n], label='class: '+str(n))
  ax.set_xlabel("Kokugo")
  ax.set_ylabel("Eigo")
  ax.set_zlabel("Suugaku")
  ax.view_init(elev=elevation)
  plt.show()
interact(polyscatter, elevation=(1, 90, 1))

図の上のバーを左右に動かすと，上下の角度が変えられます。  
とは言え，ちょっと見づらいです。  
次に，国語--英語，国語--数学，英語--数学 の3パターンで2次元プロットしてみます。  

In [ ]:
plt.figure(figsize=(18,5))
plt.subplot(1,3,1)
for n in range(num_classes):
  plt.scatter(X[y==n,0], X[y==n,1], color=color_set[n], label='class: '+str(n))
plt.xlabel('Kokugo')
plt.ylabel('Eigo')
plt.legend()
plt.subplot(1,3,2)
for n in range(num_classes):
  plt.scatter(X[y==n,0], X[y==n,2], color=color_set[n], label='class: '+str(n))
plt.xlabel('Kokugo')
plt.ylabel('Suugaku')
plt.legend()
plt.subplot(1,3,3)
for n in range(num_classes):
  plt.scatter(X[y==n,1], X[y==n,2], color=color_set[n], label='class: '+str(n))
plt.xlabel('Eigo')
plt.ylabel('Suugaku')
plt.legend()
plt.show()

左の図を見ると，**クラス0（青）は英語に比べて国語が苦手のように見えます**。  
また中央と右の図を見ると，**クラス2（緑）は数学が苦手のようです**。  

以降，圧縮手法を使って分析していきます。

## ステップ2: データの標準化
主成分分析の時と同様に，分析の前処理としてデータの標準化を行います。  

In [ ]:
# 次元毎の平均と分散
mu = np.mean(X, axis=0)
std = np.std(X, axis=0)

# 標準化
X_norm = (X - mu) / std

## ステップ3: 主成分分析を用いた場合 
ためしに，主成分分析を使って3次元データを2次元データに圧縮してプロットしてみます。  
主成分分析については05_01_pca.ipynbで解説しています。

In [ ]:
# 分散共分散行列の計算
# X.T とすることで転置してから cov に入力する点に注意
cov = np.cov(X_norm.T, bias=True)

# 固有値 eig_val と 固有ベクトル eig_vec を計算
eig_val, eig_vec =np.linalg.eig(cov)
# データによっては固有値分解結果が複素数になる場合があるため，実部のみ取得
eig_val = np.real(eig_val)
eig_vec = np.real(eig_vec)

# 固有値を大きい順に並び替える
order = np.argsort(eig_val)[::-1]
eig_val = eig_val[order]
eig_vec_pca = eig_vec[:,order]

# 第二主成分までの固有ベクトルを用いて次元圧縮
X_pca = np.dot(X_norm, eig_vec_pca[:,:2])

# 二次元データのプロット
color_set = ['b', 'r', 'g']
plt.figure(figsize=(10,5))
for n in range(num_classes):
  plt.scatter(X_pca[y==n,0], X_pca[y==n,1], color=color_set[n], label='class: '+str(n))
plt.xlabel('1st principal component')
plt.ylabel('2nd principal component')
plt.legend()
plt.show()

主成分分析を使って2次元にプロットした結果，クラス0（青）とクラス1（赤）が混ざってしまっていることが分かります。  
主成分分析はあくまでデータ全体の分散を最大化するように2次元圧縮しており，各データのクラスについては一切考慮していないため，必ずしもクラスの違いが分かるような可視化がされません。 

## ステップ4: 線形判別分析その1　クラス内分散とクラス間分散の計算
では線形判別分析を行います。  
そのための準備として，クラス内分散共分散行列`cov_inner`とクラス間分散共分散行列`cov_intra`を計算します。

In [ ]:
# クラス全体の平均
m_all = np.mean(X_norm, axis=0)

# クラス内分散共分散
cov_inner = np.zeros((num_dimensions, num_dimensions))
for n in range(num_classes):
  # クラス n に該当するデータ X_tmp
  X_tmp = X_norm[y==n]
  # クラス内平均
  m = np.mean(X_tmp, axis=0)
  cov_inner += np.dot((X_tmp-m).T, (X_tmp-m))

# クラス間分散
cov_intra = np.zeros((num_dimensions, num_dimensions))
for n in range(num_classes):
  # クラス n に該当するデータ X_tmp
  X_tmp = X_norm[y==n]
  # クラス n に該当するデータのサンプル数を得る
  num_samples_n, num_dimensions = np.shape(X_tmp)
  # クラス内平均
  m = np.mean(X_tmp, axis=0)

  # クラス間分散を計算
  m_sub = m - m_all
  m_sub = np.reshape(m_sub, (-1,1))
  cov_intra += num_samples_n * np.dot(m_sub, m_sub.T)

## ステップ5: 線形判別分析その2: クラス内/間分散共分散比行列の固有値分解  
クラス内分散の逆行列とクラス間分散の内積を計算し，計算結果に対して固有値分解を行います。  
その後，固有値の大きい順に固有ベクトルを並べ替えます。  
逆行列の計算には<font color="Red"> **`linalg.inv`**</font> 関数を使用します。

In [ ]:
# クラス内分散-クラス間分散比行列
J = np.dot(np.linalg.inv(cov_inner), cov_intra)

# 固有値 eig_val と 固有ベクトル eig_vec を計算
eig_val, eig_vec =np.linalg.eig(J)
# データによっては固有値分解結果が複素数になる場合があるため，実部のみ取得
eig_val = np.real(eig_val)
eig_vec = np.real(eig_vec)

# 固有値を大きい順に並び替える
order = np.argsort(eig_val)[::-1]
eig_val = eig_val[order]
eig_vec_lda = eig_vec[:,order]

## ステップ6: 線形判別分析その3: 次元圧縮とプロット
第一および第二固有ベクトルを使って，データを二次元に圧縮します。

In [ ]:
# 第二固有ベクトルまでとの内積を計算
X_lda = np.dot(X_norm, eig_vec_lda[:,:2])

# 二次元データのプロット
color_set = ['b', 'r', 'g']
plt.figure(figsize=(10,5))
for n in range(num_classes):
  plt.scatter(X_lda[y==n,0], X_lda[y==n,1], color=color_set[n], label='class: '+str(n))
plt.xlabel('1st component')
plt.ylabel('2nd component')
plt.legend()
plt.show()

線形判別分析によって2次元圧縮した結果は，３つのクラスがきっちり分かれて分布していることが分かります。  

## ステップ7: 考察  
線形判別分析の固有ベクトルを見てみましょう。  

In [ ]:
print(eig_vec_lda[:,:2])

一つ目の軸（横軸）は「国語：-0.86，英語：0.48，数学：0.17」となっています。  
国語が負の係数，英語が正の係数であることから，この軸では，英語が得意かつ国語が苦手なほど，正の方向に大きくなります。  
そのため横軸上では，英語に比べて国語が苦手なクラス0（青）が，他のクラスから離れます。  

二つ目の軸（縦軸）は「国語：0.44，英語：-0.49，数学：0.75」となっています。  
特に数学に対する係数が大きいため，縦軸上では，数学で特に差が顕著なクラス1(赤)とクラス3(緑)が見分けやすくなっています。